In [1]:
!pip install -U --quiet transformers accelerate
!pip install rarfile
!apt-get install -y unrar
!pip install jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.4 MB/s eta 0:00:00
Reading package lists... Done
Building depe

## Loading input


In [3]:
import json
from pandas import DataFrame
data = None
with open('predicted_data.json') as j:
  data = json.load(j)
print(len(data))
data = data[:1]
DataFrame(data).head()

50


,user_input,retrieved_contexts,response,reference
0,كتى ولد الاسكندرالاكبر؟,[الإسكندر الثالث المقدوني، المعروف بأسماء عديد...,ولد الإسكندر الأكبر في مدينة پيلا قرابة سنة 35...,قرابة سنة 356 ق.م،


## Evaluating

In [ ]:
!unzip -o "metrics_prompts.zip" -d "prompts"

Archive:  metrics_prompts.zip
  inflating: prompts/flow_judge/context_precision.txt  
  inflating: prompts/flow_judge/context_recall.txt  
  inflating: prompts/flow_judge/factual_correctness.txt  
  inflating: prompts/flow_judge/faithfulness.txt  
  inflating: prompts/flow_judge/response_relevancy.txt  
  inflating: prompts/flow_judge/_metric description prompt.txt  
  inflating: prompts/flow_judge/_prompt generating prompt.txt  
  inflating: prompts/m_promotheus/context_precision.txt  
  inflating: prompts/m_promotheus/context_recall.txt  
  inflating: prompts/m_promotheus/factual_correctness.txt  
  inflating: prompts/m_promotheus/faithfulness.txt  
  inflating: prompts/m_promotheus/response_relevancy.txt  
  inflating: prompts/m_promotheus/_prompt generating prompt for MP.txt  


In [ ]:
import zipfile
from jinja2 import Template

def extract_prompt_templates(zip_path):
    """
    Extract Jinja templates from a zip file containing prompt templates.

    Args:
        zip_path (str): Path to the zip file

    Returns:
        dict: Nested dictionary {folder_name: {metric_name: jinja2.Template}}
    """
    prompt_templates = {}
    with zipfile.ZipFile(zip_path, 'r') as z:
        for file_info in z.infolist():
            # Skip directories and hidden files
            if file_info.is_dir():
                continue

            path_parts = file_info.filename.split('/')
            # Only process files in immediate subdirectories
            if len(path_parts) != 2:
                continue

            folder, filename = path_parts
            # Skip non-target folders and hidden files
            if folder not in ['flow_judge', 'm_promotheus']:
                continue
            if filename.startswith('_') or not filename.endswith('.txt'):
                continue

            # Extract metric name (remove .txt extension)
            metric_name = filename[:-4]

            # Read and decode file content
            with z.open(file_info) as f:
                content = f.read().decode('utf-8')

            # Create Jinja template
            template = Template(content)

            # Add to nested dictionary
            if folder not in prompt_templates:
                prompt_templates[folder] = {}
            prompt_templates[folder][metric_name] = template

    return prompt_templates


In [ ]:
metrics_prompts = extract_prompt_templates('./metrics_prompts.zip')
metrics_setup_args = {'flow_judge': ['inputs', 'response'],
                      'm_promotheus': ['data', 'response', 'reference']}
eval_config = {
    'metrics': ['faithfulness', 'context_precision', 'context_recall', 'response_relevance', 'factual_correctness'],
    'default_metrics': ['faithfulness'],
    'judge': 'flowaicom/Flow-Judge-v0.1', # 'flowaicom/Flow-Judge-v0.1' (good), 'Unbabel/M-Prometheus-3B' (very bad)
    'judge_name': 'flow_judge',
    'eval_prompts': metrics_prompts,
    'eval_setup_input': metrics_setup_args,
    'batch_size': 8,
    'logging': True
}


def log_for_me(message):
  if eval_config['logging']:
    print(message)

In [ ]:
print(metrics_prompts)
# print(metrics_prompts['flow_judge']['faithfulness'].render())

{'flow_judge': {'context_precision': <Template memory:7e1876380b90>, 'context_recall': <Template memory:7e185d26b8d0>, 'factual_correctness': <Template memory:7e1876d79850>, 'faithfulness': <Template memory:7e184a845a90>, 'response_relevancy': <Template memory:7e184a844790>}, 'm_promotheus': {'context_precision': <Template memory:7e1874067010>, 'context_recall': <Template memory:7e185d97e950>, 'factual_correctness': <Template memory:7e184a77e490>, 'faithfulness': <Template memory:7e184a77eb10>, 'response_relevancy': <Template memory:7e185d244090>}}


In [ ]:
#### loading the model
from transformers import pipeline
import torch

model_id = eval_config['judge']
pipe = pipeline(
          "text-generation",
          model = model_id,
          max_new_tokens=1024,
          device="cuda",
          return_full_text=False,
          torch_dtype = torch.bfloat16,
          batch_size = eval_config['batch_size'],
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
#### setup functions
def flow_judge_data_builder(data, with_context= False):
  ready_data = []
  for row in data:
    inputs, response = "", ""

    inputs = f"""
    <user_query>
    {row['user_input']}
    </user_query>

    <reference answer>
    {row['reference']}
    </reference answer>
    """
    if with_context:
      contexts= "\n".join(row['retrieved_contexts'])
      inputs += f"""<provided information>
      {contexts}
      </provided information>"""

    response = row['response']

    ready_data.append({'inputs':inputs, 'response':response})
  return ready_data

def m_promotheus_data_builder(data, with_context= False):
  ready_data = []
  for row in data:
    data__in, response, reference = "", "", ""

    data__in = f"""
    ## user_query:
    {row['user_input']}"""

    if with_context:
      contexts = "\n".join(row['retrieved_contexts'])
      inputs += f"""## provided information to answer user query:
      {contexts}"""

    reference = row['reference']
    response = row['response']

    ready_data.append({'data':data__in,'reference':reference, 'response':response})
  return ready_data

def extract_tag_from_structured_answer(answer, begin_tag, end_tag = None, converter = None):
  if end_tag:
    val = answer.split(end_tag)[0].split(begin_tag)[1].strip()
  else:
    val = answer.split(begin_tag)[1].strip()
  if converter:
    val = converter(val)
  return val

def flow_judge_feedback_extractor(answer):
  return extract_tag_from_structured_answer(answer,'<feedback>','</feedback>')

def m_promotheus_feedback_extractor(answer):
  return extract_tag_from_structured_answer(answer,'Feedback:','[RESULT]')

def flow_judge_score_extractor(answer):
  return extract_tag_from_structured_answer(answer,'<score>','</score>')

def m_promotheus_score_extractor(answer):
  return extract_tag_from_structured_answer(answer,'[RESULT]')

judge_handlers = { 'builder' : {'flow_judge': flow_judge_data_builder,
                                'm_promotheus': m_promotheus_data_builder
                                },
                   'feedback_extractor':{'flow_judge': flow_judge_feedback_extractor,
                                'm_promotheus': m_promotheus_feedback_extractor
                                },
                   'score_extractor': {'flow_judge': flow_judge_score_extractor,
                                'm_promotheus': m_promotheus_score_extractor
                                },
}

def setup_data(data, judge):
  print(f"setting inputs for judge {judge} which are {eval_config['eval_setup_input'][judge]}'")
  return judge_handlers['builder'][judge](data)

def process_tasks(tasks, prompt_to_messages = True):
  length_of_m_tasks = len(list(tasks.values())[0])
  flattened_tasks = sum([ts for m,ts in tasks.items()],[])

  from tqdm.notebook import tqdm

  # Generate the grouped processing list
  remaining = len(flattened_tasks)
  to_processing_list = []
  start_index = 0

  # First batch: size 1
  if remaining > 0:
      batch_size = 1
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Next batch: size 2
  if remaining > 0:
      batch_size = min(2, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Next batch: size 4
  if remaining > 0:
      batch_size = min(4, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Subsequent batches: size 8 until completion
  while remaining > 0:
      batch_size = min(8, remaining)
      end_index = start_index + batch_size
      to_processing_list.append(flattened_tasks[start_index:end_index])
      start_index = end_index
      remaining -= batch_size

  # Process tasks with dynamic batch size progress
  flattened_results = []
  converter_to_messages = lambda x : [ {"role": "user", "content": x}]
  with tqdm(total=len(flattened_tasks)) as pbar:
      for batch in to_processing_list:
          batch = [converter_to_messages(t) for t in batch] if prompt_to_messages else batch
          flattened_results += (pipe(batch)) # Your processing function
          pbar.update(len(batch))

  # collect results
  results = []
  temp = []
  for res in flattened_results:
    temp.append(res)
    if len(temp) == length_of_m_tasks:
      results.append(temp)
      temp = []

  return results



def run_evaluation(data, metrics = None):
  data_as_params = setup_data(data=data, judge=eval_config['judge_name'])
  log_for_me('finished data setup')

  using_metrics = metrics or eval_config['default_metrics']
  prompts = [(m, eval_config['eval_prompts'][eval_config['judge_name']][m]) for m in using_metrics]
  prompts = dict(prompts)

  tasks = {m: [prompt.render(params) for params in data_as_params] for m, prompt in prompts.items()}
  log_for_me('finished preparing tasks')

  results = process_tasks(tasks)

  tasks_results = dict(zip(using_metrics, results))
  # extracting results
  feedback_ext = judge_handlers['feedback_extractor'][eval_config['judge_name']]
  score_ext = judge_handlers['score_extractor'][eval_config['judge_name']]
  detailed = {}
  for m, res in tasks_results.items():
    new_keys = m+'_feedback', m+'_score'
    temp = {new_keys[0]:[], new_keys[1]:[]}
    for r in res:
      answer = r[0]['generated_text']
      new_vals = feedback_ext(answer), score_ext(answer)
      temp[new_keys[0]].append(new_vals[0])
      temp[new_keys[1]].append(new_vals[1])
    detailed.update(temp)
  for m in using_metrics: # extraction is done and no need for the generated text
      tasks_results.pop(m)
  tasks_results.update(detailed)
  # preparing final output
  from pandas import DataFrame, concat
  df1,df2 = DataFrame(tasks_results), DataFrame(data)
  df = concat([df1,df2], axis = 1)
  complete_results = df.to_dict(orient = 'records')
  return complete_results


In [ ]:
## running evaluation
results = run_evaluation(data)

setting inputs for judge flow_judge which are ['inputs', 'response']'
finished data setup
finished preparing tasks


  0%|          | 0/1 [00:00<?, ?it/s]

## saving results

In [ ]:
from pandas import DataFrame
df = DataFrame(results)
df.to_json('evaluation_results__by_' + eval_config['judge_name'] + '.json', orient='records', indent=4)
print(df['retrieved_contexts'].iloc[0])
df.head()

['الإسكندر الثالث المقدوني، المعروف بأسماء عديدة أخرى أبرزها: الإسكندر الأكبر، والإسكندر الكبير، والإسكندر المقدوني، والإسكندر ذو القرنين (باليونانية: Ἀλέξανδρος ὁ Μέγας؛ نقحرة: )، هو أحد ملوك مقدونيا الإغريق، ومن أشهر القادة العسكريين والفاتحين عبر التاريخ. وُلد الإسكندر في مدينة پيلا قرابة سنة 356 ق.م، وتتلمذ على يد الفيلسوف والعالم الشهير أرسطو حتى بلغ ربيعه السادس عشر. وبحلول عامه الثلاثين، كان قد أسس إحدى أكبر وأعظم الإمبراطوريات التي عرفها العالم القديم، والتي امتدت من سواحل البحر الأيوني غربًا وصولاً إلى سلسلة جبال الهيمالايا شرقًا. يُعد أحد أنجح القادة العسكريين في مسيرتهم، إذ لم يحصل أن هُزم في أي معركة خاضها على الإطلاق.', 'الإسكندر الثالث المقدوني، المعروف بأسماء عديدة أخرى أبرزها: الإسكندر الأكبر، والإسكندر الكبير، والإسكندر المقدوني، والإسكندر ذو القرنين (باليونانية: Ἀλέξανδρος ὁ Μέγας؛ نقحرة: ألكساندروس أوميگاس)، هو أحد ملوك مقدونيا الإغريق، ومن أشهر القادة العسكريين والفاتحين عبر التاريخ.', 'خلف الإسكندر والده، فيليپ الثاني المقدوني «الأعور»، على عرش البلاد سنة 336 ق.م، 

,faithfulness_feedback,faithfulness_score,user_input,retrieved_contexts,response,reference
0,The output is mostly faithful to the input con...,4,كتى ولد الاسكندرالاكبر؟,[الإسكندر الثالث المقدوني، المعروف بأسماء عديد...,ولد الإسكندر الأكبر في مدينة پيلا قرابة سنة 35...,قرابة سنة 356 ق.م،


## Fallback Evaluations when model is not enough
using Bleu and Rouge for answer and some embedding eval for retrieval.

In [12]:
!pip install --quiet ragas sacrebleu rouge_score

  Preparing metadata (setup.py) ... done


In [25]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore
from ragas.metrics import RougeScore

from tqdm.notebook import tqdm

def traditional_evaluation(data):
  # create samples

  samples = [SingleTurnSample(response=item['response'], reference=item['reference']) for item in data]

  bleu_scorer = BleuScore()
  rouge1_scorer = RougeScore(rouge_type="rouge1")
  rougeL_scorer = RougeScore(rouge_type="rougeL")
  scores = []
  for sample in tqdm(samples):
    score = {'bleu': bleu_scorer.single_turn_score(sample),
             'rouge1': rouge1_scorer.single_turn_score(sample),
             'rougeL': rougeL_scorer.single_turn_score(sample)}
    scores.append(score)

  return scores

In [26]:
trad_results = traditional_evaluation(data)

  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
# combine with other results
from pandas import DataFrame, concat
df1,df2 = DataFrame(results), DataFrame(trad_results)
df = concat([df1,df2], axis=1)
df.to_json('evaluation_results__by_' + eval_config['judge_name'] + '.json', orient='records', indent=4)
df.head()

[{'bleu': 0.2891784933232572, 'rouge1': 1.0, 'rougeL': 1.0}]